In [1]:
import open3d as o3d
from pyntcloud import PyntCloud
import numpy as np
from sklearn.decomposition import PCA
import pandas as pd

Before voxelisation we process point cloud by remove outliers. This step is justified beacuse

In [2]:
#pcd = o3d.io.read_point_cloud("finalproject.ply")
#cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
#o3d.visualization.draw_geometries([cl])
#o3d.io.write_point_cloud("finalproject_wn.ply", cl)
pc = PyntCloud.from_file("finalproject.ply")
pcd_array = np.asarray(pc.points)
#sample = np.random.choice(pcd_array, size=(200,3,2))

In [3]:
voxelgrid_id = pc.add_structure("voxelgrid", size_x=50, size_y=50, n_z=50)
voxelgrid = pc.structures[voxelgrid_id]
voxels = voxelgrid.voxel_n

print("Number of voxels: ", len(voxels))
print("Voxel IDs: ", voxels)

# Voxels containing points
occupied_voxels = np.unique(voxelgrid.voxel_n)

# Visualize voxels
voxelgrid.plot(d=3, mode="density", cmap="hsv")

Number of voxels:  24134667
Voxel IDs:  [38974 38974 38974 ... 76771 76771 76771]


Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(24.292713165283203, 72.53204345703125, 65.68…

In [23]:
pca = PCA(n_components=3)
all_voxel_info_tmp = []


for voxel in occupied_voxels:
    ev=[]
    
    tmp = np.where(voxelgrid.voxel_n == voxel) #Gets all points that are in voxel
    voxel_points = pcd_array[tmp] #Gets all points that are in voxel
    
    xyzmax = np.array([voxel_points[:,0].max(), voxel_points[:,1].max(), voxel_points[:,2].max()])
    xyzmin = np.array([voxel_points[:,0].min(), voxel_points[:,1].min(), voxel_points[:,2].min()])
    xyzmean = np.array([voxel_points[:,0].mean(), voxel_points[:,1].mean(), voxel_points[:,2].mean()])
    num_points = len(voxel_points)
    
    linearity ,planerity, scattering, omnivariance, sum_ev, anistropy, eigentropy, curvature_change = 0,0,0,0,0,0,0,0
    
    if (len(tmp[0])) > 2: #minimum number of points to do PCA=3. Should probably exluce voxels based on N points earlier.
        pca.fit(voxel_points[:,:3])
        ev = pca.explained_variance_ratio_ #eigenvalues ratio. If you want to use sum of eigenvalues remember de_ratio.
        EV = pca.explained_variance_
        
        # Shape features
        l1, l2, l3 = ev
        L1,L2,L3 =EV
        linearity = (l1 - l2) / l1
        planerity = (l2 - l3) / l1  
        scattering = l3 / l1 
        omnivariance = (l1 * l2 * l3) ** (1 / 3) 
        sum_ev = L1 + L2 + L3 
        anistropy = (l1 - l3) / l1 
        curvature_change = l3 / sum_ev  

        if(0 not in ev):
            eigentropy = -1 * ((l1 * np.log(l1)) + (l2 * np.log(l2)) + (l3 * np.log(l3)))


        
        
    res = np.array([voxel, linearity ,planerity,scattering, omnivariance, sum_ev,anistropy, eigentropy, curvature_change, num_points])
    all_voxel_info_tmp.append(res)
        

            

        

In [24]:
tmp_arr = np.asarray(all_voxel_info_tmp)
tmp_arr.T

array([[3.84240000e+04, 3.84250000e+04, 3.84740000e+04, ...,
        7.67220000e+04, 7.67710000e+04, 7.67720000e+04],
       [3.82715918e-01, 9.70527948e-01, 1.44632562e-01, ...,
        9.55360145e-01, 2.10198376e-02, 9.33563057e-01],
       [5.17493210e-01, 2.81295533e-02, 7.46737194e-01, ...,
        3.34744209e-02, 9.41951763e-01, 5.25263866e-02],
       ...,
       [8.47993637e-01, 1.39725903e-01, 8.65800512e-01, ...,
        2.33294079e-01, 7.71975227e-01, 2.99075296e-01],
       [1.36223345e-04, 7.30871696e-05, 1.24867135e-04, ...,
        1.60764767e-04, 4.37144165e-05, 3.59307329e-04],
       [1.59800000e+04, 1.30000000e+01, 2.94090000e+04, ...,
        2.11000000e+02, 1.89280000e+04, 4.71000000e+02]])

In [26]:
#res = voxel, ev, xyzmax, xyzmin, xyzmean, num_points
arr = np.asarray(all_voxel_info_tmp)
# intialise data of lists. 
# Create DataFrame use id as index
df = pd.DataFrame(arr) 
#df.set_index(0)
df.columns = ['voxel_id', 'linearity', 'planerity', 'scattering', 'omnivariance', 'sum_ev', 'anistropy', 'eigentropy', 'curvature_change', 'num_points']
#df = df.astype({"voxel_id": int, 'num_points':int})
df = df.dropna()
#df = df.mask(df.eq('NoneType')).dropna()
df

,voxel_id,linearity,planerity,scattering,omnivariance,sum_ev,anistropy,eigentropy,curvature_change,num_points
0,38424.0,0.382716,0.517493,0.099791,0.230004,426.628658,0.900209,0.847994,0.000136,15980.0
1,38425.0,0.970528,0.028130,0.001342,0.033057,17.819368,0.998658,0.139726,0.000073,13.0
2,38474.0,0.144633,0.746737,0.108630,0.230618,442.957070,0.891370,0.865801,0.000125,29409.0
3,38475.0,0.926341,0.071406,0.002254,0.051081,83.082028,0.997746,0.264506,0.000025,138.0
4,38524.0,0.118431,0.804858,0.076711,0.208047,428.364451,0.923289,0.829395,0.000091,27849.0
...,...,...,...,...,...,...,...,...,...,...
1229,76672.0,0.965350,0.028331,0.006319,0.057901,155.419261,0.993681,0.182819,0.000039,658.0
1230,76721.0,0.126328,0.846718,0.026955,0.150810,401.198797,0.973045,0.755511,0.000035,18246.0
1231,76722.0,0.955360,0.033474,0.011165,0.075096,65.781062,0.988835,0.233294,0.000161,211.0
1232,76771.0,0.021020,0.941952,0.037028,0.164164,420.163075,0.962972,0.771975,0.000044,18928.0


In [29]:
df.plot(x='voxel_id', y='num_points')

<AxesSubplot:xlabel='voxel_id'>

AttributeError: 'NoneType' object has no attribute 'extents'

<Figure size 432x288 with 1 Axes>

Here begins the actual segmentation part. Now we have voxelized the pointcloud and the result is 475999 occupied voxels. 

False

In [ ]:
res

Now each entry in all_voxel_info_tmp -> [voxel number, [eigenvalues]]. But we want to use more information than just the result from the PCA. We can take advantage of knowledge as density, min-coords, max-coords and means. Thus we add three new fields as  -> density : dtype float, [min-coords], [max-coords], mean : float. 
Remember that this will be the cell-block above will be the main method of this project. Smarter to just do all data-processing in this loop. 

In [ ]:
density_feature_vector = voxelgrid.get_feature_vector(mode="density") #percentage of points per cloud - relative scale
len(density_feature_vector) # = density_feature_vector*len(occupied_voxels)
binary_feature_vector = voxelgrid.get_feature_vector(mode="binary")
#x_max_feature_vector = voxelgrid.get_feature_vector(mode="x_max")


In [ ]:

print(len(all_voxel_info_tmp))
 
#for i in range(len(all_voxel_info_tmp)):
    
    

In [ ]:
pc